### Import Package and set path

In [1]:
import pandas as pd
import glob
import os
from datetime import date
import shutil

In [2]:
# home and time
home = os.path.expanduser("~")
todaystr = date.today().strftime('%Y-%m-%d')

# set up concat directories
targetFolder = os.path.join(home, 'HP Inc','GPSTW SOP - 2021 日新','Project team','Upload folder ( for buyer update )')
FD_folder = os.path.join(targetFolder, "FD_today")
shortage_folder = os.path.join(targetFolder ,"shortage_today")
PNbasedDetail_folder = os.path.join(targetFolder ,"PNbasedDetail_today")

### Amend data

In [3]:
FD_amend_folder = os.path.join(targetFolder, "FD_today", 'amend')
shortage_amend_folder = os.path.join(targetFolder ,"shortage_today",'amend')
PNbasedDetail_amend_folder = os.path.join(targetFolder ,"PNbasedDetail_today",'amend')

### Function Merge and Sort

In [4]:
def merge(path: str) -> pd.DataFrame:
    # concat
    temp_file_list = []
    for f in glob.glob(path):
        # print(f)
        temp_file = pd.read_excel(f)
        temp_file_list.append(temp_file)
    All = pd.concat(temp_file_list)
    
    return All

In [5]:
def maxLen(df_all: pd.DataFrame, sort_index: list) -> pd.DataFrame:
    # sort based on len
    sort_list = []
    for _ in sort_index:
        try:
            df_all[str('len_' + _)] = df_all[_].str.len()
            sort_list.append(str('len_' + _))
        except Exception as e:
            print(e)
    df_all = df_all.reset_index(drop = True)

    max_files = []
    for i, ele in enumerate(sort_list):
        idmax = df_all[ele].max()
        max = df_all[df_all[ele] == idmax]
        max_files.append(max.head(1))
    df_max_to_add = pd.concat(max_files).drop_duplicates()

    # drop the max len row
    for i, ele in enumerate(df_max_to_add.index.values):
        df_all = df_all.drop([df_all.index[ele]])

    # concat and put on the top
    output = pd.concat([df_max_to_add, df_all]).reset_index( drop = True )

    # cut more than 500
    for _ in sort_index:
        try:
            output[_] = output[_].apply(lambda x: x[:450] if len(x) > 500 else x)
        except Exception as e:
            print(e)
    
    # final step, drop calculate step and output
    output = output.drop(columns = sort_list)
    output['Item'] = output['Item'].astype(str)

    return output

### Generate FD, shortage, PNDetail table

In [6]:
FD = merge(str(os.path.join(FD_folder,"*.xlsx")))
FD_output = maxLen(FD, ['FV','Platform'])


shortage = merge(str(os.path.join(shortage_folder,"*.xlsx")))
try:
    shortage['HP_PN'] = shortage['HP_PN'].apply(lambda x: x[:128] if len(x) > 128 else x)
except:    
    pass
Shortage_output = maxLen(shortage, ['FV','Platform'])


PN = merge(str(os.path.join(PNbasedDetail_folder,"*.xlsx")))
PNbasedDetail_output = maxLen(PN, ['GPS Remark','ODM use column1','ODM use column2','ODM use column3','ODM use column4','ODM use column5'])

Can only use .str accessor with string values!
Can only use .str accessor with string values!
object of type 'float' has no len()
object of type 'float' has no len()
object of type 'float' has no len()
object of type 'float' has no len()
object of type 'float' has no len()
object of type 'float' has no len()


### Generate amend --> FD, shortage, PNDetail table

In [7]:
# FD_amend = merge(str(os.path.join(FD_amend_folder,"*.xlsx")))
# FD_amend_output = maxLen(FD, ['FV','Platform'])


# shortage_amend = merge(str(os.path.join(shortage_amend_folder,"*.xlsx")))
# try:
#     shortage_amend['HP_PN'] = shortage_amend['HP_PN'].apply(lambda x: x[:128] if len(x) > 128 else x)
# except:    
#     pass
# Shortage_amend_output = maxLen(shortage_amend, ['FV','Platform'])


# PN_amend = merge(str(os.path.join(PNbasedDetail_amend_folder,"*.xlsx")))
# PNbasedDetail_amend_output = maxLen(PN_amend, ['GPS Remark','ODM use column1','ODM use column2','ODM use column3','ODM use column4','ODM use column5'])

### Output concated FD, Shortage, and PNbasedDetail files

In [8]:
# apache airflow to upload SQL ( currently to desktop )
FD_output.to_excel(os.path.join(home, 'Desktop', 'FD_all.xlsx'), index=False)
Shortage_output.to_excel(os.path.join(home, 'Desktop', 'Shortage_all.xlsx'), index=False)
PNbasedDetail_output.to_excel(os.path.join(home, 'Desktop', 'PNbasedDetail_all.xlsx'), index=False)

### Output concated amend FD, shortage, PNDetail

In [9]:
# FD_amend_output.to_excel(os.path.join(home, 'Desktop', 'FD_amend_all.xlsx'), index=False)
# Shortage_amend_output.to_excel(os.path.join(home, 'Desktop', 'Shortage_amend_all.xlsx'), index=False)
# PNbasedDetail_amend_output.to_excel(os.path.join(home, 'Desktop', 'PNbasedDetail_amend_all.xlsx'), index=False)

### Move file to archive

In [10]:
FD_folder = os.path.join(targetFolder, "FD_today")
FD_archive_folder = os.path.join(targetFolder, 'FD_Archive_After_1025')

for f in os.listdir(FD_folder):
    if f.endswith('.xlsx'):
        shutil.move(os.path.join(FD_folder, f), os.path.join(FD_archive_folder, f))
    else:
        pass
    
shortage_folder = os.path.join(targetFolder ,"shortage_today")
shortage_archive_folder = os.path.join(targetFolder ,"Shortage_Archive_After_1025")

for f in os.listdir(shortage_folder):
    if f.endswith('.xlsx'):
        shutil.move(os.path.join(shortage_folder, f), os.path.join(shortage_archive_folder, f))
    else:
        pass

PNbasedDetail_folder = os.path.join(targetFolder ,"PNbasedDetail_today")
PNbasedDetail_archive_folder = os.path.join(targetFolder ,"PNbasedDetail_Archive_After_1025")

for f in os.listdir(PNbasedDetail_folder):
    if f.endswith('.xlsx'):
        shutil.move(os.path.join(PNbasedDetail_folder, f), os.path.join(PNbasedDetail_archive_folder, f))
    else:
        pass


### move amend data to archive folder

In [11]:
# for f in os.listdir(FD_amend_folder):
#     shutil.move(os.path.join(FD_amend_folder, f), os.path.join(FD_archive_folder, f))

# for f in os.listdir(shortage_amend_folder):
#     shutil.move(os.path.join(shortage_amend_folder, f), os.path.join(shortage_archive_folder, f))

# for f in os.listdir(PNbasedDetail_amend_folder):
#     shutil.move(os.path.join(PNbasedDetail_amend_folder, f), os.path.join(PNbasedDetail_archive_folder, f))